In [1]:
#missing values have value ? all for attribute 11

#either edible, poisonous or unknown. Unknowm and poisonous grouped together

#to deal with missing values use 'classifier' or 'regressor' based on values that arent missing to predict missing values in
#preprocessing. Be careful that vlaues using to predict arent overfitted. Compare to other methods such as removing entire attribute
#or giving deafult/mean value


from sklearn import metrics
from sklearn import model_selection
from sklearn import ensemble
import numpy as np
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt

In [50]:
# read contents of data file
file = pd.read_csv("agaricus-lepiota.data")
  
# create list to be used as headers
features = ['edibility', 'capShape', 'capSurface', 'capColor', 'bruises', 'odor', 
            'gillAttachment', 'gillSpacing', 'gillSize', 'gillColor', 
            'stalkShape', 'stalkRoot', 'stalkSurfaceAboveRing', 'stalkSurfaceBelowRing', 
            'stalkColorAboveRing', 'stalkColorBelowRing', 'veilType', 'veilColor', 
            'ringNumber', 'ringType', 'sporePrintColor', 'population', 'habitat']
  
# converting data frame to csv
file.to_csv("agaricus-lepiota.csv", header=features, index=False)

data = pd.read_csv("agaricus-lepiota.csv")

#result = data.head(10)
#print(data)
#print(result)

In [39]:
#removes column with missing values which in our data is only stalkRoot
#one way to deal with missing values
data = data.drop('stalkRoot', axis = 1)

data.to_csv("agaricus-lepiota-no-stalkRoot.csv")

#print(data.head(10))

In [51]:
#removes all rows that have missing values
print(data.shape)
data.drop(data.loc[data['stalkRoot']=='?'].index, inplace=True)

#shows we lose alot of data as about 2500 rows lost
#bad way of handling missing values
print(data.shape)

data.to_csv("agaricus-lepiota-missing-no-?.csv")

(8123, 23)
(5643, 23)


In [ ]:
#now replace ? missing values with predicted value from a classifier
